In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pathlib import Path
from skimage.io import imshow
import matplotlib.pyplot as plt
from tqdm import tqdm

from torchvision.transforms import ToTensor, RandomCrop, Resize

from leaf.dta import LeafDataset, GetPatches, TransformPatches, test_greenness

In [ ]:
train_dset = LeafDataset("./data/train_images", "./data/train_images/labels.csv")
val_dset = LeafDataset("./data/val_images", "./data/val_images/labels.csv")

In [ ]:
img = train_dset[0][0]

In [ ]:
imshow(img)

In [ ]:
random_green = RandomGreen(64, 64, green_ratio=0.7)

In [ ]:
nongreen_imgs = []
green_imgs = []

In [ ]:
for i in tqdm(range(1000)):
    random_green(ToTensor()(img))

In [ ]:
len(green_imgs)

In [ ]:
len(nongreen_imgs)

In [ ]:
def test_greenness(img, green_tol=0.1, ratio = 0.5):
    green_mod = (img[1, :, :] + green_tol).reshape(-1)
    n_pixels = len(green_mod)
    #import pdb; pdb.set_trace()
    return ((green_mod > img[0, :, :].reshape(-1)) & (green_mod > img[2, :, :].reshape(-1))).sum() / n_pixels > ratio

In [ ]:
test_greenness(green_imgs[0])

In [ ]:
import random

In [ ]:
idx = random.randint(0, 999)
plt.imshow(green_imgs[idx].numpy().transpose([1, 2, 0]))

In [ ]:
class RandomGreen(object):
    def __init__(self, size_x, size_y, green_tol=0.1, green_ratio=0.5):
        self.size_x = size_x
        self.size_y = size_y
        self.green_tol = green_tol
        self.green_ratio = green_ratio
        self.random_crop = RandomCrop(size=(size_x, size_y), pad_if_needed=True, fill=-1)

    def __call__(self, img):
        cand = self.random_crop(img)
        while(not ((cand > -1).all() & test_greenness(cand, self.green_tol, self.green_ratio))):
            nongreen_imgs.append(cand)
            cand = self.random_crop(img)
        green_imgs.append(cand)
        return cand

In [ ]:
nongreen_imgs

In [ ]:
img = Resize((768, 576))(ToTensor()(img))

In [ ]:
get_patches = GetPatches(768, 576, 12, 9)

In [ ]:
patches = get_patches(img)

In [ ]:
plt.imshow(patches[1].numpy().transpose([1, 2, 0]))

In [ ]:
plt.imshow(img.numpy().transpose([1, 2, 0]))

In [ ]:
import torch

In [ ]:
torch.tensor([1, 2]).shape

In [ ]:
tuple(Out[112])

In [ ]:
class GetPatches(object):
    def __init__(self, img_width, img_height, x_patches, y_patches):
        assert isinstance(img_width, (int, tuple))
        assert isinstance(img_height, (int, tuple))
        assert isinstance(x_patches, (int, tuple))
        assert isinstance(y_patches, (int, tuple))
        assert img_width % x_patches == 0 and img_height % y_patches == 0
        self.x_patches = x_patches
        self.y_patches = y_patches
        self.img_width = img_width
        self.img_height = img_height
        self.patch_width = int(img_width / x_patches)
        self.patch_height = int(img_height / y_patches)

    def __call__(self, sample):
        assert tuple(sample.shape[1:]) == (self.img_width, self.img_height)
        patches = []
        for x in range(self.x_patches):
            for y in range(self.y_patches):
                patch = sample[:, (x * self.patch_width):((x + 1) * self.patch_width), (y * self.patch_height):((y + 1) * self.patch_height)]
                patches.append(patch)
        return patches